# Rf 8 0 3_Mcstudy_Addons 2
Validation and MC studies: RooMCStudy - Using the randomizer and profile likelihood add-on models



**Author:** 07/2008 - Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 03:04 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooGaussian.h"
#include "RooConstVar.h"
#include "RooChebychev.h"
#include "RooAddPdf.h"
#include "RooMCStudy.h"
#include "RooRandomizeParamMCSModule.h"
#include "RooDLLSignificanceMCSModule.h"
#include "RooPlot.h"
#include "TCanvas.h"
#include "TAxis.h"
#include "TH1.h"
#include "TDirectory.h"

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;

Create model
 -----------------------

Simulation of signal and background of top quark decaying into
 3 jets with background

Observable

In [3]:
RooRealVar mjjj("mjjj", "m(3jet) (GeV)", 100, 85., 350.);


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Signal component (gaussian)

In [4]:
RooRealVar mtop("mtop", "m(top)", 162);
RooRealVar wtop("wtop", "m(top) resolution", 15.2);
RooGaussian sig("sig", "top signal", mjjj, mtop, wtop);

Background component (chebychev)

In [5]:
RooRealVar c0("c0", "Chebychev coefficient 0", -0.846, -1., 1.);
RooRealVar c1("c1", "Chebychev coefficient 1", 0.112, -1., 1.);
RooRealVar c2("c2", "Chebychev coefficient 2", 0.076, -1., 1.);
RooChebychev bkg("bkg", "combinatorial background", mjjj, RooArgList(c0, c1, c2));

Composite model

In [6]:
RooRealVar nsig("nsig", "number of signal events", 53, 0, 1e3);
RooRealVar nbkg("nbkg", "number of background events", 103, 0, 5e3);
RooAddPdf model("model", "model", RooArgList(sig, bkg), RooArgList(nsig, nbkg));

Create manager
 ---------------------------

Configure manager to perform binned extended likelihood fits (binned(),extended()) on data generated
 with a Poisson fluctuation on Nobs (Extended())

In [7]:
RooMCStudy *mcs = new RooMCStudy(model, mjjj, Binned(), Silence(), Extended(kTRUE),
                                 FitOptions(Extended(kTRUE), PrintEvalErrors(-1)));

Customize manager
 ---------------------------------

Add module that randomizes the summed value of nsig+nbkg
 sampling from a uniform distribution between 0 and 1000

 In general one can randomize a single parameter, or a
 sum of N parameters, using either a uniform or a Gaussian
 distribution. Multiple randomization can be executed
 by a single randomizer module

In [8]:
RooRandomizeParamMCSModule randModule;
randModule.sampleSumUniform(RooArgSet(nsig, nbkg), 50, 500);
mcs->addModule(randModule);

Add profile likelihood calculation of significance. redo each
 fit while keeping parameter nsig fixed to zero. For each toy,
 the difference in -log(L) of both fits is stored, as well
 a simple significance interpretation of the delta(-logL)
 using Dnll = 0.5 sigma^2

In [9]:
RooDLLSignificanceMCSModule sigModule(nsig, 0);
mcs->addModule(sigModule);

Run manager, make plots
 ---------------------------------------------

Run 1000 experiments. this configuration will generate a fair number
 of (harmless) MINUIT warnings due to the instability of the Chebychev polynomial fit
 at low statistics.

In [10]:
mcs->generateAndFit(500);

[#0] PROGRESS:Generation -- RooMCStudy::run: sample 495
[#0] PROGRESS:Generation -- RooMCStudy::run: sample 490
[#0] PROGRESS:Generation -- RooMCStudy::run: sample 485
[#0] PROGRESS:Generation -- RooMCStudy::run: sample 480
[#0] PROGRESS:Generation -- RooMCStudy::run: sample 475
[#0] PROGRESS:Generation -- RooMCStudy::run: sample 470
[#0] PROGRESS:Generation -- RooMCStudy::run: sample 465
[#0] PROGRESS:Generation -- RooMCStudy::run: sample 460
[#0] PROGRESS:Generation -- RooMCStudy::run: sample 455
[#0] PROGRESS:Generation -- RooMCStudy::run: sample 450
[#0] PROGRESS:Generation -- RooMCStudy::run: sample 445
[#0] PROGRESS:Generation -- RooMCStudy::run: sample 440
[#0] PROGRESS:Generation -- RooMCStudy::run: sample 435
[#0] PROGRESS:Generation -- RooMCStudy::run: sample 430
[#0] PROGRESS:Generation -- RooMCStudy::run: sample 425
[#0] PROGRESS:Generation -- RooMCStudy::run: sample 420
[#0] PROGRESS:Generation -- RooMCStudy::run: sample 415
[#0] PROGRESS:Generation -- RooMCStudy::run: sam

Make some plots

In [11]:
TH1 *dll_vs_ngen = mcs->fitParDataSet().createHistogram("ngen,dll_nullhypo_nsig", -40, -40);
TH1 *z_vs_ngen = mcs->fitParDataSet().createHistogram("ngen,significance_nullhypo_nsig", -40, -40);
TH1 *errnsig_vs_ngen = mcs->fitParDataSet().createHistogram("ngen,nsigerr", -40, -40);
TH1 *errnsig_vs_nsig = mcs->fitParDataSet().createHistogram("nsig,nsigerr", -40, -40);

Draw plots on canvas

In [12]:
TCanvas *c = new TCanvas("rf803_mcstudy_addons2", "rf802_mcstudy_addons2", 800, 800);
c->Divide(2, 2);
c->cd(1);
gPad->SetLeftMargin(0.15);
dll_vs_ngen->GetYaxis()->SetTitleOffset(1.6);
dll_vs_ngen->Draw("box");
c->cd(2);
gPad->SetLeftMargin(0.15);
z_vs_ngen->GetYaxis()->SetTitleOffset(1.6);
z_vs_ngen->Draw("box");
c->cd(3);
gPad->SetLeftMargin(0.15);
errnsig_vs_ngen->GetYaxis()->SetTitleOffset(1.6);
errnsig_vs_ngen->Draw("box");
c->cd(4);
gPad->SetLeftMargin(0.15);
errnsig_vs_nsig->GetYaxis()->SetTitleOffset(1.6);
errnsig_vs_nsig->Draw("box");

Make roomcstudy object available on command line after
 macro finishes

In [13]:
gDirectory->Add(mcs);

Draw all canvases 

In [14]:
%jsroot on
gROOT->GetListOfCanvases()->Draw()